### Step 1 – Inference Using Fine-Tuned T5
We now evaluate how well the fine-tuned model performs on unseen input. 
We will tokenize the article, generate the summary, and decode the result.


In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load model from saved checkpoint
model = T5ForConditionalGeneration.from_pretrained("results/t5_finetuned")
tokenizer = T5Tokenizer.from_pretrained("results/t5_finetuned")

# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

### Step 3 – Tokenize Input Text
We provide a new input paragraph for summarization and tokenize it for model inference.


In [4]:
input_text = """summarize: Artificial Intelligence (AI) is rapidly transforming the healthcare industry. From predicting diseases to automating administrative tasks, AI is improving both patient outcomes and hospital efficiency. AI-powered tools can analyze vast amounts of medical data, assist in diagnostics, and even recommend personalized treatment plans. As technology advances, AI is expected to play an even greater role in preventive care and medical research."""

input_tokens = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)


In [6]:
output_ids = model.generate(
    input_ids=input_tokens["input_ids"],
    attention_mask=input_tokens["attention_mask"],
    max_length=100,
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=2  # Avoid repetition
)


# Decode token IDs to summary text
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Summary:", summary)


Generated Summary: AI-powered tools can analyze vast amounts of medical data, assist in diagnostics, and recommend personalized treatment plans. AI is expected to play an even greater role in preventive care and medical research.


### ✅ Final Output – Generated Summary
By adding `no_repeat_ngram_size=2`, we encouraged diversity in the model’s output, preventing repeated n-grams and ensuring richer summaries.

**Generated Summary:**
AI-powered tools can analyze vast amounts of medical data, assist in diagnostics, and recommend personalized treatment plans. AI is expected to play an even greater role in preventive care and medical research.

This confirms our fine-tuned T5 model is capable of producing coherent, meaningful summaries on unseen data.
